In [2]:
#Cross-validation and bootstrap


############################################################
#### The Validation Set Approach,   use the Auto data set.
############################################################


#### set.seed() function in order to set a seed for R's random number generator, so that every one get
####   precisely the same results as those shown below


library (ISLR)
set.seed (1)
train=sample (392 ,196)   ## randomly sample 196 observations as training data set 

lm.fit =lm(mpg~horsepower ,data=Auto ,subset =train )

attach (Auto)
mean((mpg -predict (lm.fit, Auto))[-train ]^2)  ##   test MSE


lm.fit2=lm(mpg~poly(horsepower ,2) ,data=Auto ,subset =train )    ## quadratic polynomial
mean((mpg -predict (lm.fit2 ,Auto))[-train ]^2)



lm.fit3=lm(mpg~poly(horsepower ,3) ,data=Auto ,subset =train )     ## cubic polynomial
mean((mpg -predict (lm.fit3 ,Auto))[-train ]^2)


## use a different training data set to see whether the result is the same or not


set.seed (2)
train=sample (392 ,196)
lm.fit =lm(mpg~horsepower ,subset =train)
mean((mpg -predict (lm.fit ,Auto))[-train ]^2)


lm.fit2=lm(mpg~poly(horsepower ,2) ,data=Auto ,subset =train )
mean((mpg -predict (lm.fit2 ,Auto))[-train ]^2)


lm.fit3=lm(mpg~poly(horsepower ,3) ,data=Auto ,subset =train )
mean((mpg -predict (lm.fit3 ,Auto))[-train ]^2)






The following objects are masked from Auto (pos = 4):

    acceleration, cylinders, displacement, horsepower, mpg, name,
    origin, weight, year




[1] 23.26601

[1] 18.71646

[1] 18.79401

[1] 25.72651

[1] 20.43036

[1] 20.38533

In [3]:
############################################################
#### Leave-One-Out Cross-Validation,   use the Auto data set.
############################################################



## In generalized linear model using glm() function,  cv.glm() can be used to calculate LOOCV automatically.

glm.fit=glm(mpg~horsepower ,data=Auto)   ## don't specify family="   ", then it is just the usual linear regression 
coef(glm.fit)


lm.fit =lm(mpg~horsepower ,data=Auto)
coef(lm.fit)


library (boot)   ## cv.glm() is in library boot
glm.fit=glm(mpg~horsepower ,data=Auto)
cv.err =cv.glm(Auto,glm.fit)
cv.err$delta  ## LOOCV: mean of test MSE 




(Intercept)  horsepower 
 39.9358610  -0.1578447

(Intercept)  horsepower 
 39.9358610  -0.1578447

[1] 24.23151 24.23114

In [4]:
############################################################
#### K-fold Cross-Validation,   use the Auto data set.
############################################################


## cv.glm() function can also be used to implement k-fold CV


set.seed (17)
cv.error.10= rep (0 ,10)
for (i in 1:10){
glm.fit=glm(mpg~poly(horsepower,i),data=Auto)        ## fit i-th order polynomial
cv.error.10[i]=cv.glm(Auto,glm.fit,K=10)$delta[1]    ## get K=10-fold cross-validation test MSE for each i-th polynomial model
}
cv.error.10


[1] 24.27207 19.26909 19.34805 19.29496 19.03198 18.89781 19.12061 19.14666
 [9] 18.87013 20.95520

In [5]:
############################################################
#### Bootstrap,   use the Portfolio data set.
############################################################



##  consider estimate variance of alpha defined in lecture notes.


## define a function to calcualte hat(alpha)

alpha.fn=function(data,index){
X=data$X[index]
Y=data$Y[index]
return((var(Y)-cov(X,Y))/(var(X)+var(Y) -2*cov(X,Y)))
}



## apply the function to Portfolio data set

alpha.fn(Portfolio,1:100)


set.seed (1)
alpha.fn(Portfolio,sample(100 ,100 , replace =T))    ## randomly sample 100 data with replacement from the data set with 100 observations.

## we need to repeat the above sampling B=1000 times.


boot(Portfolio,alpha.fn,R=1000)     ## use boot() function to implement bootstrap.  R is the number of repeatition



[1] 0.5758321

[1] 0.7368375


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Portfolio, statistic = alpha.fn, R = 1000)


Bootstrap Statistics :
     original       bias    std. error
t1* 0.5758321 -0.001695873  0.09366347

In [6]:
############################
####  use boobtrap to linear regression model: estimate the variance of coefficient estimate. Auto data is used.
##########################


boot.fn=function(data,index){
return (coef(lm(mpg~horsepower ,data=data ,subset =index)))
}

boot.fn(Auto ,1:392)


set.seed (1)
boot.fn(Auto ,sample (392 ,392 , replace =T))    ## output coefficient estimate for a resampled data set.
boot.fn(Auto ,sample (392 ,392 , replace =T))	 ## output coefficient estimate for a resampled data set.


boot(Auto ,boot.fn ,1000)     ## bootstrap estimate of the standard error of the coefficient estimate.



summary (lm(mpg~horsepower ,data=Auto))$coef    ## compare bootstrap estimate of SE with that in lm() .



## consider quadratic model.

boot.fn=function (data ,index )
coefficients(lm(mpg~horsepower +I( horsepower ^2) ,data=data ,subset =index))
set.seed (1)
boot(Auto ,boot.fn ,1000)


summary(lm(mpg~horsepower +I(horsepower ^2) ,data=Auto))$coef     ### now the two estimates of SE are closer.


(Intercept)  horsepower 
 39.9358610  -0.1578447

(Intercept)  horsepower 
 40.3404517  -0.1634868

(Intercept)  horsepower 
 40.1186906  -0.1577063


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Auto, statistic = boot.fn, R = 1000)


Bootstrap Statistics :
      original        bias    std. error
t1* 39.9358610  0.0544513229 0.841289790
t2* -0.1578447 -0.0006170901 0.007343073

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),39.9358610,0.717498656,55.65984,1.220362e-187
horsepower,-0.1578447,0.006445501,-24.48914,7.031989e-81



ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Auto, statistic = boot.fn, R = 1000)


Bootstrap Statistics :
        original        bias     std. error
t1* 56.900099702  3.511640e-02 2.0300222526
t2* -0.466189630 -7.080834e-04 0.0324241984
t3*  0.001230536  2.840324e-06 0.0001172164

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),56.900099702,1.8004268063,31.60367,1.740911e-109
horsepower,-0.466189630,0.0311246171,-14.97816,2.289429e-40
I(horsepower^2),0.001230536,0.0001220759,10.08009,2.196340e-21
